In [1]:
import requests
from lxml import etree
import re

ModuleNotFoundError: No module named 'lxml'

In [2]:
def extract_content_with_cookie():
    # 配置请求参数
    url = "https://aws.highspot.com/spots/60bdbd9634d6be4dbd9ce328"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
        'Cookie': 'viewer2=v2-eyJ1dWlkIjoiMjdmYjgwN2MtZDdiMy00ZTQwLTg5OGYtN2VjYmU3NDNmNmQy%0AIn0%3D%0A; hs_app=1; __reveal_ut=d38f106e-4027-48c6-5d61-88aa729c4d38; rack.session2=eyJzZXNzaW9uX2lkIjoiOTA3YWNmNzUyNWU2ZTViMTU4YmM5MGE3MGZmMjE1ZTU1Zjk4YmQzZmFjYmQ5ZjkzNDYyMzUxNjVlZTM3ODFkNSIsIl9mbGFzaCI6e30sIm51dGVsbGEiOiI2NzIwODliNjJlZTlmOTAyYjAwNjhlOGQiLCJzdSI6InN1MCIsInRva2VuIjoiMjo6ZTdlODYzNzYtZmRkOS00NWIzLWFiYzYtM2E1MzVjMjRlY2FjOjAiLCJjc3JmIjoiVk1mczVyc20wVTlwc3ZnLVRSRk9HekpMQkxLaTg5WVFtUk5aTC1LNmdqQSJ9--42cbf63d49db7524c8954488c010fc9fb6f655ee; amp_27c1db=RhA8KNWbXXrmYx0EeEtak4.MTI1ZmJjYWUwYjU2M2NjZWE5MTQ=..1ilfpd11o.1ilfpd11p.h.3m.47'  # 替换为实际获取的cookie
    }

    try:
        # 发送请求
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()  # 自动处理4xx/5xx错误
        print(response.text)

        # 解析HTML
        res = response.text
        html = etree.HTML(response.text)
        
        # 构建优化的XPath（两类定位方案）
        xpaths = [
            # 精确类名匹配方案
            '//div[contains(@class, "Thumbnail-module__image") and contains(@class, "Thumbnail-module__landscape")]',
            # 结构定位备用方案
            '//main//article//div[contains(@class, "div")]//div[@style]'
        ]

        found_elements = []
        for xpath in xpaths:
            elements = html.xpath(xpath)
            if elements:
                found_elements = elements
                break

        if not found_elements:
            raise ValueError("未找到目标元素，请检查：1. Cookie有效性 2. 页面结构是否变更")

        # 提取URL
        url_pattern = re.compile(r'url\(["\']?(.*?)["\']?\)')  # 支持单/双引号和无引号格式
        results = []

        for element in found_elements:
            style = element.get('style', '')
            if not style:
                continue
            
            match = url_pattern.search(style)
            if match:
                # 提取并清理URL
                raw_url = match.group(1)
                clean_url = raw_url.split('?')[0].replace('&amp;', '&')
                results.append(clean_url)

        # 输出结果
        print(f"共找到 {len(results)} 个背景图片URL：")
        for i, url in enumerate(results, 1):
            print(f"[URL {i}] {url}")

    except requests.exceptions.RequestException as e:
        print(f"网络请求失败：{str(e)}")
    except Exception as e:
        print(f"程序异常：{str(e)}")
        return res

In [3]:
res = ""
res = extract_content_with_cookie()

<!DOCTYPE html><html class='desktop  ' lang='en'><head>
<meta charset='utf-8'>
<meta content='IE=edge;chrome=1' http-equiv='X-UA-Compatible'>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.init={"loader":"spa","distributed_tracing":{"enabled":true,"exclude_newrelic_header":false,"cors_enabled":false,"cors_use_newrelic_header":false,"cors_use_tracecontext_headers":false,"allowed_origins":[]},"privacy":{"cookies_enabled":true},"ajax":{"deny_list":["bam.nr-data.net"]},"pinned_version":null,"session_trace":{"enabled":false,"sampling_rate":0,"error_sampling_rate":0},"session_replay":{"enabled":false,"autoStart":true,"block_selector":"","mask_text_selector":"*","mask_all_inputs":true,"sampling_rate":0.0,"error_sampling_rate":0.0,"collect_fonts":true,"inline_images":false,"inline_stylesheet":true}};NREUM.loader_config={"accountID":"450341","agentID":"2594769","applicationID":"2594760","trustKey":"450341","licenseKey":"8e4fd79c2e"};NREUM.info={"applicationID":"2594760","beacon":

In [4]:
fout = open("output.html", "w")
fout.write(res)
fout.close()